## TensorBoard support
TensorBoard is a dashboard that comes with TensorFlow. It the helps visualizing NN graphs as well as all kinds of metrics around training and performance.

So far, the only metric we looked at as the win-loss-draw ratio out NN achieved. By using TensorBoard we might be able to get some additional insights.

In order to do this, we need to create a summary writer, write to it, and in the end close it:

```Python
sess = TFSessionManager.get_session()
writer = tf.summary.FileWriter(TENSORLOG_DIR, sess.graph)
[...]
writer.close()
```

To capture the actual metrics, we add the approptiate lines when building graph;
```Python
def build_graph(self, name: str):
    """
    Builds a new TensorFlow graph with scope `name`
    :param name: The scope for the graph. Needs to be unique for the session.
    """
    with tf.variable_scope(name):
        [...]
        self.q = tf.reduce_sum(tf.multiply(self.q_values, self.actions_onehot), axis=1, name="selected_action_q")

        tf.summary.histogram("Action Q values", self.q)

        [...]
        self.loss = tf.reduce_mean(self.td_error, name="q_loss")

        tf.summary.scalar("Q Loss", self.loss)
        [...]
        reg_loss = self.beta * tf.reduce_mean(self.reg_losses)
        tf.summary.scalar("Regularization loss", reg_loss)

        self.merge = tf.summary.merge_all()

        [...]

```

This creates a Tensor `self.merge` which we can run with the training step:

```Python
def final_result(self, result: GameResult):
    [...]
    
    summary, _ = TFSN.get_session().run([self.q_net.merge, self.q_net.train_step], feed_dict=[...])
    
    [...]
    
    if self.writer is not None:
        self.writer.add_summary(summary, self.game_counter)
        summary = tf.Summary(value=[tf.Summary.Value(tag='Random Move Probability',
                                                     simple_value=self.random_move_prob)])
        self.writer.add_summary(summary, self.game_counter)
```